<a href="https://colab.research.google.com/github/Kajecik/prirlab9prog2/blob/main/program2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

!nvcc --version
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git



%load_ext nvcc_plugin

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'nvidia-kernel-common-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-325-updates' for glob 'nvidia*'
Note, selecting 'nvidia-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-binary' for glob 'nvidia*'
Note, selecting 'nvidia-331-dev' for glob 'nvidia*'
Note, selecting 'nvidia-304-updates-dev' for glob 'nvidia*'
Note, selecting 'nvidia-compute-utils-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-384-dev' for glob 'nvidia*'
Note, selecting 'nvidia-libopencl1-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-fs-prebuilt' for glob 'nvidia*'
Note, selecting 'nvidia-driver-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-340-updates-uvm' for glob 'nvidia*'
Note, selecting 'nvidia-dkms-450-server' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-source-440-server' for glob 'nvidia*'


In [2]:
%%cu
#include <stdio.h>
#define N 10000

clock_t start = clock();

void add( int *a, int *b, int *c ) {
int tid = 0; // to jest CPU nr zero, a więc zaczynamy od zera
while (tid < N) {
c[tid] = a[tid] + b[tid];
tid += 1; // mamy tylko jeden CPU, a więc zwiększamy o jeden
}
}
int main( void ) {
int a[N], b[N], c[N];
// zapelnianie tablic a i b za pomocą CPU
for (int i=0; i<N; i++) {
a[i] = -i;
b[i] = i * i;
}
add( a, b, c );
// wyswietlenie wynikow
for (int i=0; i<N; i++) {
printf( "%d + %d = %d\n", a[i], b[i], c[i] );
}

clock_t end = clock();
float seconds = (float)(end - start) / CLOCKS_PER_SEC;

printf("\n %f", seconds);

return 0;


}

Strumieniowane dane wyjściowe obcięte do 5000 ostatnich wierszy.
-5002 + 25020004 = 25015002
-5003 + 25030009 = 25025006
-5004 + 25040016 = 25035012
-5005 + 25050025 = 25045020
-5006 + 25060036 = 25055030
-5007 + 25070049 = 25065042
-5008 + 25080064 = 25075056
-5009 + 25090081 = 25085072
-5010 + 25100100 = 25095090
-5011 + 25110121 = 25105110
-5012 + 25120144 = 25115132
-5013 + 25130169 = 25125156
-5014 + 25140196 = 25135182
-5015 + 25150225 = 25145210
-5016 + 25160256 = 25155240
-5017 + 25170289 = 25165272
-5018 + 25180324 = 25175306
-5019 + 25190361 = 25185342
-5020 + 25200400 = 25195380
-5021 + 25210441 = 25205420
-5022 + 25220484 = 25215462
-5023 + 25230529 = 25225506
-5024 + 25240576 = 25235552
-5025 + 25250625 = 25245600
-5026 + 25260676 = 25255650
-5027 + 25270729 = 25265702
-5028 + 25280784 = 25275756
-5029 + 25290841 = 25285812
-5030 + 25300900 = 25295870
-5031 + 25310961 = 25305930
-5032 + 25321024 = 25315992
-5033 + 25331089 = 25326056
-5034 + 25341156 = 25336122
-5035 + 253

In [3]:
%%cu
#include <stdio.h>
#define N 10000

clock_t start = clock();

__global__ void add( int *a, int *b, int *c ) {
int tid = blockIdx.x; // dodawanie na danych znajdujących się pod indeksem
if (tid < N)
c[tid] = a[tid] + b[tid];
}

int main( void ) {
 int a[N], b[N], c[N];
 int *dev_a, *dev_b, *dev_c;
 // alokacja pamięci na GPU
cudaMalloc( (void**)&dev_a, N * sizeof(int) );
cudaMalloc( (void**)&dev_b, N * sizeof(int) );
cudaMalloc( (void**)&dev_c, N * sizeof(int) );
 // zapelnienie tablic a i b na CPU
 for (int i=0; i<N; i++) {
 a[i] = -i;
 b[i] = i * i;
 }
// kopiowanie tablic a i b do GPU
cudaMemcpy( dev_a, a, N * sizeof(int), cudaMemcpyHostToDevice );
cudaMemcpy( dev_b, b, N * sizeof(int), cudaMemcpyHostToDevice );
 add<<<N,1>>>( dev_a, dev_b, dev_c );
 // kopiowanie tablicy c z GPU do CPU
cudaMemcpy( c, dev_c, N * sizeof(int), cudaMemcpyDeviceToHost );
 // wyświetlenie wynikow
 for (int i=0; i<N; i++) {
 printf( "%d + %d = %d\n", a[i], b[i], c[i] );
 }
 // zwolnienie pamieci alokowanej na CPU
 cudaFree( dev_a );
 cudaFree( dev_b );
 cudaFree( dev_c );

clock_t end = clock();
float seconds = (float)(end - start) / CLOCKS_PER_SEC;

printf("\n%f", seconds);

 return 0;
}

Strumieniowane dane wyjściowe obcięte do 5000 ostatnich wierszy.
-5002 + 25020004 = 25015002
-5003 + 25030009 = 25025006
-5004 + 25040016 = 25035012
-5005 + 25050025 = 25045020
-5006 + 25060036 = 25055030
-5007 + 25070049 = 25065042
-5008 + 25080064 = 25075056
-5009 + 25090081 = 25085072
-5010 + 25100100 = 25095090
-5011 + 25110121 = 25105110
-5012 + 25120144 = 25115132
-5013 + 25130169 = 25125156
-5014 + 25140196 = 25135182
-5015 + 25150225 = 25145210
-5016 + 25160256 = 25155240
-5017 + 25170289 = 25165272
-5018 + 25180324 = 25175306
-5019 + 25190361 = 25185342
-5020 + 25200400 = 25195380
-5021 + 25210441 = 25205420
-5022 + 25220484 = 25215462
-5023 + 25230529 = 25225506
-5024 + 25240576 = 25235552
-5025 + 25250625 = 25245600
-5026 + 25260676 = 25255650
-5027 + 25270729 = 25265702
-5028 + 25280784 = 25275756
-5029 + 25290841 = 25285812
-5030 + 25300900 = 25295870
-5031 + 25310961 = 25305930
-5032 + 25321024 = 25315992
-5033 + 25331089 = 25326056
-5034 + 25341156 = 25336122
-5035 + 253